In [13]:
import concurrent.futures
from datetime import datetime
import requests
from bs4 import BeautifulSoup
from openpyxl import Workbook
from google.colab import files
import ipywidgets as widgets
from IPython.display import display, clear_output
import time
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry

# Variables globales
proceso_en_ejecucion = False
estado_codigos = []
total_codigos = 0
codigos_procesados = 0
start_time = None

def requests_retry_session(retries=3, backoff_factor=0.3, status_forcelist=(500, 502, 504)):
    session = requests.Session()
    retry = Retry(
        total=retries,
        read=retries,
        connect=retries,
        backoff_factor=backoff_factor,
        status_forcelist=status_forcelist,
    )
    adapter = HTTPAdapter(max_retries=retry)
    session.mount('http://', adapter)
    session.mount('https://', adapter)
    return session

def obtener_estado_precio_imagenes(codigo_padre, pais):
    if not codigo_padre:
        return "Código vacío", "N/A", 0, "N/A"

    session = requests_retry_session()

    try:
        # Obtener estado
        url_estado = f'https://www.marathon.store/{pais}/view/ProductVariantSelectorComponentController?componentUid=VariantSelector&currentProductCode={codigo_padre}'
        response_estado = session.get(url_estado, timeout=10)
        estado = "Agotado"
        if response_estado.status_code == 200:
            soup_estado = BeautifulSoup(response_estado.content, 'html.parser')
            lis = soup_estado.find_all('li', attrs={'data-url': lambda x: x and str(codigo_padre) in x})
            if lis:
                for li in lis:
                    data_has_stock = li.get('data-has-stock')
                    data_selected = li.get('data-selected')
                    if data_has_stock == "true" or data_selected == "true":
                        estado = "Disponible"
                        break
        elif response_estado.status_code == 404:
            estado = "ERROR 404"
        else:
            estado = f"Error {response_estado.status_code}: No se pudo conectar"

        # Obtener precio e imágenes desde la página del producto
        url_precio = f'https://www.marathon.store/{pais}/p/{codigo_padre}'
        response_precio = session.get(url_precio, timeout=10)
        precio = "N/A"
        imagenes = []

        if response_precio.status_code == 200:
            soup_precio = BeautifulSoup(response_precio.content, 'html.parser')

            # Buscar el precio utilizando el selector del div contenedor (según el XPath compartido)
            precio_element = soup_precio.select_one('div.desktop-price')  # Actualiza según la clase real del precio

            if not precio_element:
                precio_element = soup_precio.select_one('div.price')
            if not precio_element:
                precio_element = soup_precio.select_one('[itemprop="price"]')

            if precio_element:
                precio = precio_element.text.strip()
                # Limpiar el precio de cualquier texto adicional
                precio = ''.join(filter(lambda x: x.isdigit() or x in [',', '.'], precio))

            # Buscar todas las imágenes dentro del div con clase 'desktop-image-gallery'
            galeria_imagenes = soup_precio.find('div', class_='desktop-image-gallery')
            if galeria_imagenes:
                imagenes = [img['data-src'] for img in galeria_imagenes.find_all('img', attrs={'data-src': True})]
            cantidad_imagenes = len(imagenes)
            enlaces_imagenes = ', '.join(imagenes)

        else:
            print(f"Error al obtener el precio para el código {codigo_padre}: Estado HTTP {response_precio.status_code}")

        return estado, precio, cantidad_imagenes, enlaces_imagenes
    except Exception as e:
        print(f"Error al obtener los datos para el código {codigo_padre}: {e}")
        return f"Error: {e}", "N/A", 0, "N/A"

# Función para actualizar el progreso
def actualizar_progreso():
    global codigos_procesados, total_codigos, start_time
    tiempo_transcurrido = datetime.now() - start_time
    horas, resto = divmod(tiempo_transcurrido.seconds, 3600)
    minutos, segundos = divmod(resto, 60)
    clear_output(wait=True)
    print(f"Progreso: {codigos_procesados}/{total_codigos}")
    print(f"Tiempo transcurrido: {horas:02d}:{minutos:02d}:{segundos:02d}")

# Función para procesar los códigos y guardar en Excel
def procesar_codigos(codigos, pais):
    global proceso_en_ejecucion, estado_codigos, total_codigos, codigos_procesados, start_time
    total_codigos = len(codigos)
    codigos_procesados = 0
    start_time = datetime.now()

    def obtener_estado_precio_imagenes_concurrente(codigo_padre, pais):
        global codigos_procesados
        estado, precio, cantidad_imagenes, enlaces_imagenes = obtener_estado_precio_imagenes(codigo_padre, pais)
        codigos_procesados += 1
        actualizar_progreso()
        return estado, precio, cantidad_imagenes, enlaces_imagenes

    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = {executor.submit(obtener_estado_precio_imagenes_concurrente, codigo, pais): codigo for codigo in codigos}
        for future in concurrent.futures.as_completed(futures):
            codigo = futures[future]
            try:
                estado, precio, cantidad_imagenes, enlaces_imagenes = future.result()
            except Exception as exc:
                estado, precio, cantidad_imagenes, enlaces_imagenes = f"Error: {exc}", "N/A", 0, "N/A"
            estado_codigos.append((codigo, estado, precio, cantidad_imagenes, enlaces_imagenes))

    if proceso_en_ejecucion:
        guardar_resultados(pais)

# Función para guardar los resultados en Excel y descargar el archivo
def guardar_resultados(pais):
    global proceso_en_ejecucion
    proceso_en_ejecucion = False

    # Guardar en Excel
    wb = Workbook()
    ws = wb.active
    ws.title = "Control Stock Web"
    ws['A1'] = "CODIGO"
    ws['B1'] = "STATUS WEB"
    ws['C1'] = "PRECIO"
    ws['D1'] = "CANTIDAD IMÁGENES"
    ws['E1'] = "ENLACES IMÁGENES"
    for i, (codigo, estado, precio, cantidad_imagenes, enlaces_imagenes) in enumerate(estado_codigos, start=2):
        ws[f'A{i}'] = codigo
        ws[f'B{i}'] = estado if estado else "Estado no disponible"
        ws[f'C{i}'] = precio
        ws[f'D{i}'] = cantidad_imagenes
        ws[f'E{i}'] = enlaces_imagenes

    # Guardar archivo
    fecha_actual = datetime.now().strftime("%Y-%m-%d")
    nombre_archivo = f"Control Stock Web {pais.upper()} {fecha_actual}.xlsx"
    wb.save(nombre_archivo)
    files.download(nombre_archivo)

# Widgets de entrada en Colab
codigos_text = widgets.Textarea(
    value='',
    placeholder='Ingrese los códigos separados por espacio',
    description='Códigos:',
    layout={'width': '500px', 'height': '100px'}
)

pais_selector = widgets.Dropdown(
    options=[('Perú', 'pe'), ('Ecuador', 'ec'), ('Bolivia', 'bo'), ('Chile', 'ch')],
    value='pe',
    description='País:'
)

start_button = widgets.Button(description="Iniciar Procesamiento")
stop_button = widgets.Button(description="Detener")

display(codigos_text, pais_selector, start_button, stop_button)

# Función para iniciar el procesamiento
def iniciar_procesamiento(b):
    global proceso_en_ejecucion
    if not proceso_en_ejecucion:
        proceso_en_ejecucion = True
        pais = pais_selector.value
        codigos = codigos_text.value.split()
        procesar_codigos(codigos, pais)

# Función para detener el procesamiento
def detener_procesamiento(b):
    global proceso_en_ejecucion
    proceso_en_ejecucion = False
    guardar_resultados(pais_selector.value)

start_button.on_click(iniciar_procesamiento)
stop_button.on_click(detener_procesamiento)


Progreso: 71/71
Tiempo transcurrido: 00:00:26


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>